<a href="https://colab.research.google.com/github/swanny0000/ZTM-TensorFlow/blob/master/08_Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NLP (or sequence-to-sequence) can be used for:
* Classification
* Generation
* Machine Translation
* Voice Assistants (deriving meaning out of speech sound waves)

Generally, these are *sequence* problems, which can have a few different forms, including:
* One to many (intake an image, output a multi-word description)
* Many to one (given a textual response, generate a sentiment e.g. positive or negative)
* Time-series (estimate the price of stock in the future)
* Many to many (translating languages)


##What we'll cover
* downloading and preparing a text dataset
* tokenization and embedding
* set up multiple recurrent neural networks
* build a feature extraction model using Tensorflow hub
and more!

In [1]:
# check for GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-9fde74e7-c766-54b4-2b30-d7f26c6f40c7)


In [2]:
# Helper functions!
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import the functions
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-09-14 04:17:30--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2024-09-14 04:17:30 (15.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# get text dataset from Kaggle's intro to NLP dataset (text samples of Tweets labelled as disaster or not disaster) - https://www.kaggle.com/c/nlp-getting-started
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip the dataset
unzip_data("nlp_getting_started.zip")

--2024-09-14 04:17:36--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 142.250.141.207, 74.125.137.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2024-09-14 04:17:36 (116 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
#let's become one with the data - visualize!
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


We'll mainly care about the "Text" and "Target" columns

In [5]:
# shuffle the dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# our test dataset doesn't have labels though
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# how many examples?
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [8]:
# how many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
Worried about how the CA drought might affect you? Extreme Weather: Does it Dampen Our Economy? http://t.co/fDzzuMyW8i

---

Target: 0 (not real disaster)
Text:
@YoungHeroesID Lava Blast &amp; Power Red #PantherAttack @JamilAzzaini @alifaditha

---

Target: 1 (real disaster)
Text:
Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM: Investigators and the families of those who were... http://t.co/yi54XRHQGB

---

Target: 1 (real disaster)
Text:
Our builder is having a dental emergency. Which has ruined my plan to emotionally blackmail him this afternoon with my bump.

---

Target: 1 (real disaster)
Text:
BMX issues Areal Flood Advisory for Shelby [AL] till Aug 5 9:00 PM CDT http://t.co/62OddEkVLi

---



In [10]:
# let's create a validation set from our training data
from sklearn.model_selection import train_test_split

In [11]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # 10% of data for validation
                                                                            random_state=42)

In [12]:
# check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# check first ten examples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [18]:
# need to tokenize our sentences!
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [21]:
text_vectorizer = TextVectorization(max_tokens=None, # how many words are in the vocabulary
                                    standardize="lower_and_strip_punctuation", # remove the fluff
                                    split="whitespace", # split up by word
                                    ngrams=None, # create groups of N words
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # what's the max length we want to use

In [22]:
# find the average number of tokens in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [23]:
# set up text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length of our sequences (e.g. how many words does a model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [30]:
# try it myself using the adapt function
text_vectorizer.adapt(train_sentences)
text_vectorizer.get_vocabulary()[:30]

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'be',
 'are',
 'was',
 'have',
 'like',
 'as',
 'up',
 'so',
 'just']

In [31]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

note the padded 0s at the end, and shape is 1 because there are fewer than 15 words

In [32]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Malaysian prime minister says Reunion Island wreckage is from MH370: http://t.co/bpTZAMjl2K via @slate      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1817, 2017, 2043,  296,  432,  419,  356,    9,   20,  171,    1,
          49, 3406,    0,    0]])>

In [35]:
# how many unique words do we have? hint... we already set that!

top_5_words = text_vectorizer.get_vocabulary()[:5]
bottom_5_words = text_vectorizer.get_vocabulary()[-5:]
print(len(text_vectorizer.get_vocabulary()))
print(top_5_words)
print(bottom_5_words)

10000
['', '[UNK]', 'the', 'a', 'in']
['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


why don't we create our own embedding layer that can learn on it's own, alongside the model?

## Creating an Embedding Layer
* [TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)

The parameters we care most about are:
* `input_dim` - the size of our vocabulary
* `output_dim` - the size of our embedding vector
* `input_length` - length of sequences being passed to embedding layer

In [37]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128) # hooray, divisible by 8!
embedding

<Embedding name=embedding_1, built=False>

In [38]:
# embed a random sentence
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Crying out for more! Set me ablaze      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02450735,  0.00085908,  0.01861769, ...,  0.03152036,
         -0.00019316, -0.03626107],
        [-0.02742995, -0.04537872, -0.04779477, ..., -0.02862008,
         -0.04390204, -0.02100347],
        [-0.02433479,  0.02541438, -0.03791391, ...,  0.04399851,
          0.02476369, -0.04490266],
        ...,
        [ 0.02977926,  0.01125325,  0.04288396, ..., -0.02778488,
         -0.01032795,  0.04953961],
        [ 0.02977926,  0.01125325,  0.04288396, ..., -0.02778488,
         -0.01032795,  0.04953961],
        [ 0.02977926,  0.01125325,  0.04288396, ..., -0.02778488,
         -0.01032795,  0.04953961]]], dtype=float32)>

In [41]:
# check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.02450735,  0.00085908,  0.01861769, -0.01493448, -0.00153899,
         0.01828606,  0.03659585,  0.02387415,  0.01493038, -0.01401902,
        -0.03613637, -0.03793547,  0.01376612, -0.02889859, -0.04296495,
        -0.00063454,  0.01916449, -0.000744  , -0.01545541,  0.03908725,
        -0.03689911,  0.02532704, -0.02130961, -0.04048084,  0.03750584,
         0.04597421,  0.02033151, -0.00761721,  0.03467375,  0.04434607,
         0.03575737, -0.04650393,  0.0278019 ,  0.02787656,  0.01568708,
        -0.04473562,  0.01763746, -0.00731559,  0.01824394,  0.03377457,
        -0.03715109, -0.04930909,  0.0060079 , -0.00520132,  0.04711342,
         0.03638751,  0.03171741,  0.00357945, -0.02316727, -0.03475165,
         0.01122897,  0.00074508, -0.04156455, -0.00540043, -0.0244045 ,
         0.01798523, -0.02195956, -0.00588766,  0.04674443,  0.00013026,
        -0.03777206,  0.04407034,  0.0132683 , -0.00317857, -0.00931028,
  

## UPCOMING TESTING
1. Naive Bayes with TF-IDF encoder (baseline)
1. Feed-forward neural network
1. LSTM (RNN)
1. GRU (RNN)
1. Bidirectional LSTM (RNN)
1. 1D Convolutional Neural Network
1. TensorFlow Hub Pretrained Feature Extractor
1. TensorFlow Hub Pretrained Feature Extractor, but 10% of data
